In [ ]:
#MATLAB CODE
'''
function [sys,x0,str,tss]=cstr1(t,x,u,flag,Param,X_ss)

global d cp V CAin DH1 DH2 UA k01 k02 EA1 EA2 Tin F Tu CDin PA PB PC PD PHEAT y0 r X0;

%
% An s-function to model a simple CSTR with first order kinetics and the following
% reaction ( A + D --> B) and ( A --> C) with a variable height well mixed vessel.  The equations that describe
% the system are:
% dV/dt  = F_in - F
% R1     = k1*Cd*Ca
% R2     = k2*Ca
% dCa/dt = F_in/V*(Ca_in-Ca)-R1-R2
% dCb/dt = F_in/V*(Cb_in-Cb)+R1
% dCc/dt = F_in/V*(Cc_in-Cc)+R2
% dCd/dt = F_in/V*(Cd_in-Cd)-R1
% dT /dt = - T/V*(F_in - F) + 1/V *(F_in*T_in - F*T) + 1/d/Cp*(-DH1)*R1+ 1/d/Cp*(-DH2)*R2 + 1/V/d/Cp*U*A*(Tc - T)

%================================================================================

switch flag,

case 0,	% Initialize the states and sizes
   [sys,x0,str,tss] = mdlInitialSizes(t,x,u,X_ss);
   
        % ****************
  	%  Outputs
  	% ****************   
     
case 3,   % Calculate the outputs

sys = mdlOutputs(t,x,u,Param);
   
   % ****************
   % Update
   % ****************
case 1,	% Obtain derivatives of states
   
   sys = mdlDerivatives(t,x,u,Param);

otherwise,
   sys = [];
end

% ******************************************
% Sub-routines or Functions
% ******************************************

% ******************************************
% Initialization
% ******************************************
function [sys,x0,str,tss] = mdlInitialSizes(t,x,u,X_ss);

global d cp V CAin DH1 DH2 UA k01 k02 EA1 EA2 Tin F Tu CDin PA PB PC PD PHEAT y0 r X0;

% This handles initialization of the function.
% Call simsize of a sizes structure.
sizes = simsizes;
sizes.NumContStates  = 6;     % continuous states
sizes.NumDiscStates  = 0;     % discrete states
sizes.NumOutputs     = 7;     % outputs of model (manipulated variables)
sizes.NumInputs      = 9;     % inputs of model [r;Y;Ud]
sizes.DirFeedthrough = 1;     % System is not causal, there is explicit dependence of outputs on inputs.
sizes.NumSampleTimes = 1;     %
sys = simsizes(sizes);        %
x0  = X_ss;                   % Initialize the discrete states (inputs).
str = [];	                  % set str to an empty matrix.
tss = [0,0];	              % sample time: [period, offset].

% ******************************************
%  Outputs
% ******************************************
function sys = mdlOutputs(t,x,u,Param);

global d cp V CAin DH1 DH2 UA k01 k02 EA1 EA2 Tin F Tu CDin PA PB PC PD PHEAT y0 r X0;


SWITCH = Param(1);
% entering flow
F_in = u(1)+5;
% exiting volumetric flow
F    =u(7)+5;

% outputs:
sys(1) = x(1);
sys(2) = F;
sys(3) = x(2);
sys(4) = x(3);
sys(5) = x(4);
sys(6) = x(5);
sys(7) = x(6);
% sys(8) = (PB*F*x(3)+PC*F*x(5)-PA*F*(u(2)+1)-PD*F*(u(8)+1)-PHEAT*(UA*((u(4)+200)-x(4))));

% ******************************************
% Derivatives
% ******************************************
function sys = mdlDerivatives(t,x,u,Param)

global d cp V CAin DH1 DH2 UA k01 k02 EA1 EA2 Tin F Tu CDin PA PB PC PD PHEAT y0 r X0;

% External Parameters:
% [0;1;-1000;1;1;25;21500;10;-1000;91500],

SWITCH = Param(1);
k01    = u(9)+1;
DH1    = Param(3);
d      = Param(4);
Cp     = Param(5);
UA     = Param(6);
EA1    = Param(7);
k02    = Param(8);
DH2    = Param(9);
EA2    = Param(10);

% Inputs (Disturbances):
% entering flow
F_in = u(1)+5;
% Entering concentration of A:
Ca   = u(2)+1;
% Entering concentration of B:
Cb   = u(3);
% Coolant Temperature in Jacket:
Tc   = u(4)+200;
% Entering concentration of C:
Cc   = u(5);

% Inputs (Manipulated Variables):
% exiting volumetric flow
F    = u(7)+5;
% Feed Temperature
T_in = u(6)+140;

% Entering Concentration of D
Cd = u(8)+1;

% Derivatives:
V = x(1);
k1 = k01 * exp (-EA1/8.314/(1.8*x(4)+1000));
k2 = k02 * exp (-EA2/8.314/(2.8*x(4)+1000));
R1 = k1*x(6)*x(2);
R2 = k2*x(2);
sys(1) = F_in - F;
sys(2) = F_in/V*(Ca-x(2))-R1-R2;
sys(3) = F_in/V*(Cb-x(3))+R1;
sys(4) = - x(4)/V*(F_in - F) + 1/V *(F_in*T_in - F*x(4)) + 1/d/Cp*(-DH1)*R1 + 1/d/Cp*(-DH2)*R2 + 1/V/d/Cp*UA*(Tc - x(4));
sys(5) = F_in/V*(Cc-x(5))+R2;
sys(6) = F_in/V*(Cd-x(6))-R1;
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


In [ ]:
def cstr_model(x,t,parameters):
    #reaction ( A + D --> B) and ( A --> C)
    k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in  = parameters
    
   
    
    k1 =k01 * np.exp (-EA1/8.314/(1.8*x[3]+1000))        
    k2 =k02 * np.exp (-EA2/8.314/(2.8*x[3]+1000))        
    
    
  
    V= x[0]
    Ca= x[1]
    Cb= x[2]
    T= x[3]
    Cc= x[4]
    Cd= x[5]
    
    R1 = k1 *Ca*Cd
    R2 = k2*Ca
    
    
    dVdt  =  F_in-F
    dCadt = (F_in/V*(Ca_in-Ca))-R1-R2
    
    dCbdt =(F_in/V*(Cb_in-Cb))+R1
    
    dTdt = - T/V*(F_in - F) + 1/V *(F_in*T_in - F*T) + 1/d/Cp*(-DH1)*R1 + 1/d/Cp*(-DH2)*R2 + 1/V/d/Cp*UA*(Tc - T)
    
    dCcdt = (F_in/V*(Cc_in- Cc))+R2
    dCddt=(F_in/V*(Cd_in-Cd))-R1
   
    
    
    return [  dVdt, dCadt,dCbdt,dTdt,dCcdt,dCddt] #COnc out

    

In [ ]:
def pp_random_walk():
    dims = 1
    step_n = 4000
    step_set = [-1, 0, 1]
    origin = np.zeros((1,dims))
    # Simulate steps in 1D
    step_shape = (step_n,dims)
    steps = np.random.choice(a=step_set, size=step_shape)
    path = np.concatenate([origin, steps]).cumsum(0)
    
    return path
def av(path, fluc_price):
    
 
    x1 = path[:1000]
    x2 = path[1000:2000]
    x3 = path[2000:3000]
    x4 = path[3000:]
    catch = [x1,x2,x3,x4]
    con = []
    for i in range(len(catch)):
        y = 0
        for i2 in range(len(catch[i])):
            y += catch[i][i2]
        av = y/len(catch[i])
        con.append(av)
    
    #print(con)

    maxr = 0
    for i in range(len(con)):
        if abs(maxr) < abs(con[i]):
            maxr = con[i]
    #print(maxr)
    
    if fluc_price > maxr:
        range_ = fluc_price/maxr 
        pl = np.ones(4000)
        pl[:1000] = (con[0] *range_ )
        pl[1000:2000] = (con[1]*range_ )
        pl[2000:3000] = (con[2]*range_ )
        pl[3000:] = (con[3]*range_ )
    else:
        print(maxr)
        maxr = maxr*-1
        range_ =fluc_price/maxr #maxr /fluc_price
        pl = np.ones(4000)
        pl[:1000] = (con[0] *range_ )
        pl[1000:2000] = (con[1]*range_ )
        pl[2000:3000] = (con[2]*range_ )
        pl[3000:] = (con[3]*range_ )
    
        
    return pl
        


In [ ]:
def av1(path, fluc_price):
    
    x1 = path[:1333]
    x2 = path[1333:2666]
    x3 = path[2666:]
 
    catch = [x1,x2,x3]
    con = []
    for i in range(len(catch)):
        y = 0
        for i2 in range(len(catch[i])):
            y += catch[i][i2]
        av = y/len(catch[i])
        con.append(av)
    
    #print(con)

    maxr = 0
    for i in range(len(con)):
        if abs(maxr) < abs(con[i]):
            maxr = con[i]
    #print(maxr)
    
    if fluc_price > maxr:
        range_ = fluc_price/maxr
        pl = np.ones(4000)
        pl[:1333] = (con[0] *range_ )
        pl[1333:2666] = (con[1]*range_ )
        pl[2666:] = (con[2]*range_ )
      
    else:
        #maxr = maxr*-1
        range_ =maxr /fluc_price
        pl = np.ones(4000)
        pl[:1333] = (con[0] /range_ )
        pl[1333:2666] = (con[1]/range_ )
        pl[2666:] = (con[2] /range_ )
       
    
        
    return pl
        


In [ ]:

import numpy as np
import random

In [ ]:
class ChemEnv():
    def __init__(self,disturbance): #initalise with the disturbances and add them to simulate_react at certain timesteps
       
       
        self.T_disturbance = disturbance[0]
        self.Ca_disturbance = disturbance[4]
        self.p_disturbance = [disturbance[1],disturbance[2],disturbance[3]]
        
        self.state = None
        self.done = False
        
        #self.episode_length= 4000
    def step(self, action,episode_timestep,old_inputs,concs_T_V,prices,bounds): #what we do when we take a step
        new_inputs = simulate_inputs(self.reset,action,episode_timestep,old_inputs, self.T_disturbance, self.Ca_disturbance,bounds)
        k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in = new_inputs
        
        new_concs_T_V = simulate_react(self.reset,episode_timestep,new_inputs,concs_T_V)
        
        self.current_V = new_concs_T_V[0][episode_timestep]
        self.current_Ca = new_concs_T_V[1][episode_timestep]
        self.current_Cb = new_concs_T_V[2][episode_timestep]
        self.current_T = new_concs_T_V[3][episode_timestep]
        self.current_Cc = new_concs_T_V[4][episode_timestep]
        self.current_Cd = new_concs_T_V[5][episode_timestep]
        profitt, Ca_p, Cb_p, Cc_p = profit_(new_concs_T_V,prices,episode_timestep, self.p_disturbance, new_inputs)
        
        self.episode_timestep = episode_timestep
        self.current_Tc = Tc[episode_timestep]
        self.current_T_in = T_in[episode_timestep]
        self.current_Ca_price = Ca_p
        self.current_Cb_price = Cb_p
        self.current_Cc_price = Cc_p
        self.current_Ca_in = Ca_in[episode_timestep]
        self.current_Cd_in = Cd_in[episode_timestep]
        self.current_F_in = F_in[episode_timestep]
   
        if self.episode_timestep == int(3998):
            self.done = True
            print(self.reward)
        else:
            self.done = False
            profit =profitt
        
        done = self.done
        self.reward += profitt
        self.reward1 = profitt
      
        self.state = ( new_concs_T_V,new_inputs, profitt  )
        #set placeholder for info
        info = {}
        
        #return step information
        return self.state, self.reward1, done, info
        
    def render(self): #visualisations
        #implement visualisation
        #get all current values,write to file and visualise in different python script
        with open("ml2txt.txt", "a") as myfile:
            myfile.write(f'\n{self.current_Tc},{self.current_Ca},{self.current_Cb},{self.current_T},{self.current_Cc},{self.current_Cd},{self.current_T_in},{self.current_Ca_price},{self.current_Cb_price},{self.current_Cc_price}, {self.reward},{self.current_Ca_in},{self.current_Cd_in},{self.current_F_in}')
            myfile.close()
        #pass
    def reset(self): #how we reset our enviroment
        self.reward =0
        self.reward1 =0
        input_reset = simulate_inputs(True,1,1,1,1,1,1  )
        concs_T_V = simulate_react(True, 0,input_reset,1)
        V, Ca, Cb, T, Cc, Cd = concs_T_V
        #clear contents in text file
        file = open("ml2txt.txt","r+")
        file.truncate(0)
        file.close()
        self.state = [[V, Ca, Cb,T,Cc, Cd], input_reset]
        return self.state

In [ ]:
def simulate_inputs(reset,action, timestep, inputs,disturbance, disturbance_, bounds):
    t = np.linspace(0,4000,4000)#starting,end,how many points(from 0 to end) i used 4000 because number of episodes per batch

    if reset == True: 

        

        k01=np.ones(len(t))*1
        DH1=np.ones(len(t))*-1000
        d =np.ones(len(t))*1
        Cp=np.ones(len(t))*1
        UA=np.ones(len(t))*25
        EA1=np.ones(len(t))*21500
        k02=np.ones(len(t))*10
        DH2=np.ones(len(t))*-1000
        EA2=np.ones(len(t))*91500
        F_in=np.ones(len(t))*5
        Ca_in=np.ones(len(t))*1
        Cb_in =np.ones(len(t))*0
        Tc = np.ones(len(t))*200
        Cc_in = np.ones(len(t))*0
        F = np.ones(len(t))*5
        T_in = np.ones(len(t))*140
        Cd_in = np.ones(len(t))*1
        return [k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in]
    else:
       
        k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in = inputs
     
         
        Ca_in[timestep:] = 1  + disturbance_[timestep]  
        Cd_in[timestep:] = min(bounds[2],max(bounds[3], Cd_in[timestep-1:timestep] + action[0])) 
        Tc[timestep:] = min(bounds[4],max(bounds[5],Tc[timestep-1:timestep] + (action[1]))) 
        F_in[timestep:] = min(bounds[0],max(bounds[1],F_in[timestep-1:timestep] + (action[2])))
        F[timestep: ] =  F_in[timestep:]
        T_in[timestep:] = 140 +  disturbance[timestep] 
        return [k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in]
        

In [ ]:
def simulate_react(reset,episode_timestep,inputs,concs_T_V):
    t = np.linspace(0,4000,4000)
    if reset == True:
        i = episode_timestep
        V0 = 100
        Ca0 = 0.3038
        Cb0 = 0.534
        T0 = 306
        Cc0 = 0.162
        Cd0 = 0.466
        V  = np.ones(len(t))*V0
        Ca = np.ones(len(t))*Ca0
        Cb = np.ones(len(t))*Cb0
        T  = np.ones(len(t))*T0
        Cc = np.ones(len(t))*Cc0
        Cd = np.ones(len(t))*Cd0
        k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in = inputs
        input_ = [k01[i],DH1[i],d[i],Cp[i],UA[i],EA1[i],k02[i],DH2[i],EA2[i],F_in[i],Ca_in[i],Cb_in[i],Tc[i],Cc_in[i],F[i],T_in[i],Cd_in[i] ]
       
        x0 = [V0,Ca0,Cb0,T0,Cc0,Cd0]
        ts = [i, i+1]
        y = odeint(cstr_model,x0,ts,args=(input_,))
        # Store results
        V[i+1]  = y[-1][0]
        Ca[i+1] = y[-1][1]
        Cb[i+1] = y[-1][2] 
        T[i+1]  = y[-1][3]
        Cc[i+1]  = y[-1][4]
        Cd[i+1]  = y[-1][5]
        dChangedt = y[-1]
        concs_T_V = V,Ca,Cb,T,Cc,Cd 
        return concs_T_V #[dChangedt]
    else:
        i = episode_timestep
        k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in = inputs
        input_ = [k01[i],DH1[i],d[i],Cp[i],UA[i],EA1[i],k02[i],DH2[i],EA2[i],F_in[i],Ca_in[i],Cb_in[i],Tc[i],Cc_in[i],F[i],T_in[i],Cd_in[i] ]
        
        V,Ca,Cb,T,Cc,Cd = concs_T_V
        x = [V[i],Ca[i],Cb[i],T[i],Cc[i],Cd[i]]
        ts = [i, i+1]
        y = odeint(cstr_model,x,ts,args=(input_,))
        # Store results
        V[i+1]  = y[-1][0]
        Ca[i+1] = y[-1][1]
        Cb[i+1] = y[-1][2] 
        T[i+1]  = y[-1][3]
        Cc[i+1]  = y[-1][4]
        Cd[i+1]  = y[-1][5]
        dChangedt = y[-1]
        concs_T_V = V,Ca,Cb,T,Cc,Cd  
        
        return concs_T_V 


In [ ]:
def profit_(concs_T_V, prices, episode_timestep, price_disturbance, new_inputs):
    i = episode_timestep
    V,Ca,Cb,T,Cc,Cd = concs_T_V
    price_Ca,price_Cb,price_Cc,price_Cd,price_heat= prices
    disturb_Ca, disturb_Cb, disturb_Cc  =price_disturbance
    k01,DH1,d,Cp,UA,EA1,k02,DH2,EA2,F_in,Ca_in,Cb_in,Tc,Cc_in,F,T_in,Cd_in = new_inputs
    
    F_in = F_in[i]
    UA =25
    Ca_in = Ca_in[i]
    Cd_in = Cd_in[i]
    Cb = Cb[ i]
    T = T[i]

    Cc = Cc[ i]
    price_Ca = price_Ca + disturb_Ca[i]
    price_Cb = price_Cb + disturb_Cb[i]
    price_Cc = price_Cc + disturb_Cc[i]
    
    temp_cost = (price_heat*UA*(Tc[i]-T))
    Ca_cost = (price_Ca* Ca_in)
    Cd_cost = (price_Cd* Cd_in)
    delta_Tc = 200-Tc[i]
    penalty_T1 = (100*tf.math.tanh(T-800))+100  
    penalty_T2 = (100*tf.math.tanh(-T+100))+100 
    penalty_CC = (100*tf.math.tanh(-Cc+0.1))+100
    penalty_CB = (100*tf.math.tanh(-Cb+0.1))+100
    total_penalty =  penalty_T1 +  penalty_T2  + penalty_CC+ penalty_CB
 

    
    
    profit = F_in*(((Cb*price_Cb)+ (Cc*price_Cc)) -((price_Ca* Ca_in)+(price_Cd* Cd_in))) + temp_cost
    
    
    
    profit_ = (profit)-(0.1*total_penalty)
   
    return [profit_,price_Ca,price_Cb,price_Cc]
    
    
   

In [ ]:
def disturbance_generator(episode, number_of_episodes):
    prices = [5, 12, 19, 2, 0.001]
    i1 = 0.1
    i2 = 0.01
    i3 = 0.3
    
    power = (episode/number_of_episodes)*10
    
    ca_disturb_bound = 1*((1+i2)**power)
    ca_disturb_bound = min(ca_disturb_bound,2)
    cb_disturb_bound = 1*((1+i1)**power)
    cb_disturb_bound = min(cb_disturb_bound,8)
    cc_disturb_bound = 1*((1+i1)**power)
    cc_disturb_bound = min(cc_disturb_bound,9)
    cc_disturb_bound *=-1
    ca_in_disturb_bound =0.15*((1+i3)**power)
    ca_in_disturb_bound = min( ca_in_disturb_bound,0.4)
    ###T-disturbance
    sr=0.05
    ts = 1.0/sr
    t = np.arange(0,80000,ts)
    freq = 0.00004 
    y = 60*np.sin(2*np.pi*freq*t)
    T_disturb = y
    #######Ca_disturb price
    t1 = pp_random_walk()
    Ca_disturb_price = av1(t1,ca_disturb_bound) #2
    ######Cb_disturb price
    t2 = pp_random_walk()
    Cb_disturb_price = av1(t2,cb_disturb_bound) #8
    ######Cc_disturb price
    t3 = pp_random_walk()
    Cc_disturb_price = av1(t3, cc_disturb_bound ) #-9
    ######Ca_in disturb
    t4 = pp_random_walk()
    Ca_in_disturb = av(t4, ca_in_disturb_bound)
    
    disturbs = [T_disturb,Ca_disturb_price,Cb_disturb_price,Cc_disturb_price,Ca_in_disturb]
    
    return disturbs

In [ ]:
def test_generator(disturb):
    ############################################
    T_disturb = disturb[0]
    Ca_disturb_price = disturb[1]
    Cb_disturb_price = disturb[2]
    Cc_disturb_price = disturb[3]
    Ca_in_disturb = disturb[4]
    #############################################
    Ca_in_disturb[0:500] = 0 #1000
    Ca_in_disturb[500:2000] = 0.4#0.3 1000-2200
    Ca_in_disturb[2000:] = 0.05#0.1 2200
    #############################################
    Ca_disturb_price[0:1500] = 0 #1200
    Ca_disturb_price[1500:]=1.5#2 1200
    #############################################
    Cb_disturb_price[0:1100]= 0
    Cb_disturb_price[1100:]= 5#8
    #############################################
    Cc_disturb_price[0:1000]= 0 #420
    Cc_disturb_price[1000:]= -8#-9 #420
    ############################################
    
    return [T_disturb,Ca_disturb_price,Cb_disturb_price,Cc_disturb_price,Ca_in_disturb]

In [ ]:
def bounds_generator(episode, number_of_episodes):
    upper_F = 15
    lower_F = 12
    upper_Cd = 0.6
    lower_Cd = 0.2
    upper_T = 375
    lower_T = 350
    upper_r = 0.5
    lower_r = 0.6
 
    power = (episode/number_of_episodes)*10
    upper_F = upper_F*((1+upper_r)**power)
    upper_F = min(25,upper_F)
    upper_Cd = upper_Cd*((1+upper_r)**power)
    upper_Cd = min(5,upper_Cd)
    upper_T = upper_T*((1+upper_r)**power)
    upper_T = min(550,upper_T)
    #################################################
    lower_F = lower_F*((1+lower_r)**-power)
    lower_F = max(1,lower_F)
    lower_Cd = lower_Cd*((1+lower_r)**-power)
    lower_Cd = max(0.001,lower_Cd)
    lower_T =lower_T*((1+lower_r)**-power)
    lower_T = max(280,lower_T)
    
    return [upper_F,lower_F,upper_Cd,lower_Cd,upper_T,lower_T] #[upper_F,lower_F,upper_Cd,lower_Cd,upper_T,lower_T]#[25,1,5,0.001,550,280]

In [ ]:

import import_ipynb
from ipynb.fs.full.S_A_C_1A import Agent
import numpy as np

In [ ]:
#!pip install ipynb

In [ ]:
import math as m
agent = Agent()
score_catcher = []
avg_catcher = []
agg = []

In [ ]:
import tensorflow as tf

In [ ]:
agent.load_models()


In [ ]:

prices = [5, 12, 19, 2, 0.001] #Ca,Cb,Cc,Cd,Energy
r = []
r1 = []
score = 0
max_ = 0
number_episodes =250
alpha_beg = 0.99
ir = 0.2
for epi in range(number_episodes):
    bounds = bounds_generator(epi,number_episodes)
    if epi == 0:
        disturbs = disturbance_generator(epi,number_episodes)
        #test_disturb = test_generator(disturbs)
        #env = ChemEnv(test_disturb)
        env = ChemEnv(disturbs)
    elif epi % 15 == 0:
        disturbs = disturbance_generator(epi,number_episodes)
        #test_disturb = test_generator(disturbs)
        #env = ChemEnv(test_disturb)
        env = ChemEnv(disturbs) #new disturbances
    else:
        pass #same disturbances as before
    timesteps = 4000
    current_state =env.reset()
    obs = np.array([current_state[0][1], current_state[0][2],current_state[0][3],current_state[0][4],current_state[0][5],current_state[1][-2],(5+disturbs[1][0])/5,(12+disturbs[2][0])/12,(19+disturbs[3][0])/19,current_state[1][-5],current_state[1][-1],current_state[1][-7]])#.astype('float32') 
    start = True
    score = 0
    alpha = alpha_beg*((1+ir)**((-epi/number_episodes)*10))
    for timestep in range(1,timesteps-1):
        action = agent.choose_action([obs],timestep,start)
        step = env.step(action,timestep,current_state[1],current_state[0],prices,bounds)
        env.render()
        current_state = [step[0][0],step[0][1]]

        obs_ = np.array([current_state[0][1], current_state[0][2],current_state[0][3],current_state[0][4],current_state[0][5], current_state[1][-2],(5+disturbs[1][timestep])/5, (12+disturbs[2][timestep])/12,(19+disturbs[2][timestep])/19,current_state[1][-5],current_state[1][-1],current_state[1][-7]]) #current_state[1][-2]#.astype('float32')
        s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12  = obs
        obs_r = np.array([s1[timestep], s2[timestep], s3[timestep]/1000, s4[timestep], s5[timestep],s6[timestep]/1000,s7,s8,s9,s10[timestep]/1000,s11[timestep],s12[timestep]])
        s1a,s2a,s3a,s4a,s5a,s6a,s7a,s8a,s9a, s10a,s11a,s12a  = obs_
        obs_r_ = np.array([s1a[timestep], s2a[timestep], s3a[timestep]/1000, s4a[timestep], s5a[timestep],s6a[timestep]/1000,s7a,s8a,s9a,s10a[timestep]/1000,s11a[timestep],s12a[timestep]])
        reward = step[1]
        done = step[2]
        agent.remember(obs_r, action, reward, obs_r_, done)
        obs = obs_
        score += reward
        start = False
        r.append(score)
       
    agent.learn(alpha)
    
    r1.append(score)
    avg_s = np.mean(r[-4000:])
    avg_catcher.append(avg_s)
 
    print(avg_s)




In [ ]:
agent.save_models()


In [ ]:
file = open('mltxt.txt', 'w')
file.close()